In [1]:
import sys
print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(["/home/mathia/PycharmProjects/neslearn-git/"])

Python 3.4.2 (default, Oct  8 2014, 10:45:20) 
[GCC 4.9.1] on linux


In [2]:
% load_ext autoreload

In [3]:
%autoreload

In [5]:
# change into a sub dir to make file spam less anoying
%cd a_star

/home/mathia/PycharmProjects/neslearn-git/hsa/tree_search/a_star


In [4]:
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
from hsa.machine_constants import mario_rom_location
from hsa.tree_search import heuristics
from collections import namedtuple
import heapq
from hsa.tree_search.heap import Heap
import pandas
import math

In [6]:
nes = NESInterface(mario_rom_location,eb_compatible=False)

In [7]:
# nes.act(0)
# nes.render()
nes.reset_game()
nes.render()

In [25]:
actions = nes.getMinimalActionSet()
breath = len(actions)
depth = 32
action_repeat = 40
master_heuristic = heuristics.dont_die

In [9]:
MarioNode = namedtuple("MarioNode",["id","potential","recv_reward"])

In [10]:
def id_to_filename(id):
    return "a{}".format(id).encode()

In [11]:
render_search = True
class AStarGraph(object):
    # Define a class board like grid with two barriers

    def __init__(self,max_heurisic = 14):
        self.max_heurisic = max_heurisic

    def heuristic(self, start):
        return max_heurisic - start.potential 

    def get_vertex_neighbours(self, pos):
        n = []
        for action in actions:
            nes.restoreShapshot(id_to_filename(pos.id))
            reward_for_action = sum([nes.act(action) for i in range(action_repeat)])
            if render_search:
                nes.render()
            new_id = hash((pos.id,action))
            new_potential = master_heuristic(nes.getRAM())
            nes.getSnapshot(id_to_filename(new_id))
            n.append(MarioNode(id=new_id,potential=new_potential,recv_reward=reward_for_action))
        return n

    # should this be the reward?
    def move_cost(self, a, b):
        for barrier in self.barriers:
            if b in barrier:
                return 100  # Extremely high cost to enter barrier squares
        return 1  # Normal movement cost

Look at the AStarGraph

In [12]:
def make_start(reset = False):
    if reset:
        nes.reset_game()
        for i in range(160):
            nes.act(0)
    reward = nes.act(0)
    nes.getSnapshot(id_to_filename("astart"))
    potential = master_heuristic(nes.getRAM())
    return MarioNode(id="astart",potential=potential,recv_reward=reward)

In [13]:
start_node = make_start(True)
nes.render()
start_node

MarioNode(id='astart', potential=40, recv_reward=0)

In [14]:
graph = AStarGraph()

In [127]:
graph.get_vertex_neighbours(start_node)

[MarioNode(id=-4600203454125191374, potential=40, recv_reward=0),
 MarioNode(id=-4600203454126273899, potential=40, recv_reward=0),
 MarioNode(id=-4600203454127356424, potential=40, recv_reward=0),
 MarioNode(id=-4600203454107870974, potential=40, recv_reward=0),
 MarioNode(id=-4600203454263754574, potential=42, recv_reward=10),
 MarioNode(id=-4600203454194472974, potential=37, recv_reward=-25),
 MarioNode(id=-4600203454159832174, potential=40, recv_reward=15),
 MarioNode(id=-4600203454108953499, potential=40, recv_reward=0),
 MarioNode(id=-4600203454264837099, potential=42, recv_reward=10),
 MarioNode(id=-4600203454195555499, potential=35, recv_reward=-35),
 MarioNode(id=-4600203454160914699, potential=40, recv_reward=25),
 MarioNode(id=-4600203454110036024, potential=40, recv_reward=0),
 MarioNode(id=-4600203454265919624, potential=42, recv_reward=10),
 MarioNode(id=-4600203454196638024, potential=36, recv_reward=-30),
 MarioNode(id=-4600203454161997224, potential=40, recv_reward=20)

In [30]:
%timeit graph.get_vertex_neighbours(start_node)

10 loops, best of 3: 92.6 ms per loop


In [38]:
# is it faster in the temp dir
here = !pwd
%mkdir /tmp/mathia_treesearch/
%cd /tmp/mathia_treesearch/
%timeit graph.get_vertex_neighbours(start_node)
%cd {here[0]}
# doesn't seem like it

mkdir: das Verzeichnis „/tmp/mathia_treesearch/“ kann nicht angelegt werden: Die Datei existiert bereits
/tmp/mathia_treesearch
10 loops, best of 3: 98.4 ms per loop
[Errno 2] No such file or directory: 'here'
/tmp/mathia_treesearch


In [15]:
def render_node(node):
    nes.restoreShapshot(id_to_filename(node.id))
    nes.render()

## More Primitiv Best First Search

In [16]:
nr_nodes_to_search = 100
seen_nodes = []
def best_first(start, graph):
    # this potential transformation should probably be some where else
    candidates = Heap([start],lambda node: -math.floor(node.potential/25))
    best_seen = MarioNode(id=None,potential=0,recv_reward=0)
    for i in range(nr_nodes_to_search):
        best_candidate = candidates.pop()
        for child in graph.get_vertex_neighbours(best_candidate):
            candidates.push(child)
            seen_nodes.append(child)
            if child.potential > best_seen.potential:
                best_seen = child
    return best_seen
            

In [26]:
search_result = best_first(start_node,graph)
search_result

MarioNode(id=4745316327722219435, potential=2593, recv_reward=60)

In [31]:
render_node(search_result)
for i in range(30):
    nes.act(0)
    nes.render()

In [19]:
pandas.DataFrame(seen_nodes)["id"].duplicated().value_counts()

False    1500
Name: id, dtype: int64

## A Star Search

In [ ]:
def a_star_search(start, graph):
    G = {}  # Actual movement cost to each position from the start position
    F = {}  # Estimated movement cost of start to end going via this position

    # Initialize starting values
    G[start] = 0
    F[start] = graph.heuristic(start)

    closedVertices = set()
    openVertices = set([start])
    cameFrom = {}

    while len(openVertices) > 0:
        # Get the vertex in the open list with the lowest F score
        current = None
        currentFscore = None
        for pos in openVertices:
            if current is None or F[pos] < currentFscore:
                currentFscore = F[pos]
                current = pos

                # Check if we have reached the goal
        if current == end:
            # Retrace our route backward
            path = [current]
            while current in cameFrom:
                current = cameFrom[current]
                path.append(current)
            path.reverse()
            return path, F[end]  # Done!

            # Mark the current vertex as closed
        openVertices.remove(current)
        closedVertices.add(current)

        # Update scores for vertices near the current position
        for neighbour in graph.get_vertex_neighbours(current):
            if neighbour in closedVertices:
                continue  # We have already processed this node exhaustively
            candidateG = G[current] + graph.move_cost(current, neighbour)

            if neighbour not in openVertices:
                openVertices.add(neighbour)  # Discovered a new vertex
            elif candidateG >= G[neighbour]:
                continue  # This G score is worse than previously found

                # Adopt this G score
            cameFrom[neighbour] = current
            G[neighbour] = candidateG
            H = graph.heuristic(neighbour, end)
            F[neighbour] = G[neighbour] + H

    raise RuntimeError("A* failed to find a solution")

### Old code for refrence

In [23]:
def search():
    best_action = 0
    best_reward = 0
    reward_sum = 0
    for itteration in range(depth):
        nes.getSnapshot(b"search")
        for action in actions:
            nes.restoreShapshot(b"search")
            current_reward = sum([nes.act(action) for i in range(action_repeat)])
            if current_reward > best_reward:
                best_action = action
                best_reward = current_reward
        nes.restoreShapshot(b"search")
        #print("Action: {}, Reward: {}".format(best_action,best_reward))
        for i in range(action_repeat):
            reward_sum += nes.act(best_action)
            nes.render()
        best_action = 0
        best_reward = 0
    return reward_sum

In [24]:
nes.reset_game()
search()

1950

In [16]:
search()

0

### TODO

- consider makeing the actions stocastic
- fix value while dying